In [2]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\Joaquín Amat\Documents\GitHub\skforecast


In [3]:
import numpy as np
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

# Fixtures
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import y

In [4]:
rng = np.random.default_rng(12345)
y = pd.Series(
    data=rng.normal(loc=10, scale=1, size=201),
    index=pd.date_range(start="01-01-2000", periods=201, freq="h"),
)
forecaster = ForecasterAutoreg(
    regressor=LinearRegression(), lags=1, binner_kwargs={"n_bins": 2}
)
forecaster.fit(y)   
X_train, y_train = forecaster.create_train_X_y(y=y)
y_pred = forecaster.regressor.predict(X_train)
for i in range(1, 20):
    forecaster.set_out_sample_residuals(y_true=y_train, y_pred=y_pred, append=True)
    for v in forecaster.out_sample_residuals_by_bin_.values():
        print(min(1_000, 100 * i))


100
100
200
200
300
300
400
400
500
500
600
600
700
700
800
800
900
900
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


In [6]:
np.array([1]) + np.array([1])

array([2])

In [55]:
rng = np.random.default_rng(12345)
y = pd.Series(
    data=rng.normal(loc=10, scale=1, size=1001),
    index=pd.date_range(start="01-01-2000", periods=1001, freq="h"),
)
forecaster = ForecasterAutoreg(
    regressor=LinearRegression(), lags=1, binner_kwargs={"n_bins": 2}
)
forecaster.fit(y)   
X_train, y_train = forecaster.create_train_X_y(y=y)
y_pred = forecaster.regressor.predict(X_train)

for i in range(1, 20):
    print(i)
    forecaster.set_out_sample_residuals(y_true=y_train, y_pred=y_pred, append=True)
    for v in forecaster.out_sample_residuals_by_bin_.values():
        #assert len(v) == min(1_000, 100 * i)
        print(len(v), min(5_000, 500 * i))

1
500 500
500 500
2
1000 1000
1000 1000
3
1500 1500
1500 1500
4
2000 2000
2000 2000
5
2500 2500
2500 2500
6
3000 3000
3000 3000
7
3500 3500
3500 3500
8
4000 4000
4000 4000
9
4500 4500
4500 4500
10
5000 5000
5000 5000
11
5000 5000
5000 5000
12
5000 5000
5000 5000
13
5000 5000
5000 5000
14
5000 5000
5000 5000
15
5000 5000
5000 5000
16
5000 5000
5000 5000
17
5000 5000
5000 5000
18
5000 5000
5000 5000
19
5000 5000
5000 5000


In [81]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from skforecast.preprocessing import RollingFeatures
from skforecast.ForecasterAutoreg import ForecasterAutoreg

# Fixtures
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import y
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import exog
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import exog_predict

In [92]:
rng = np.random.default_rng(12345)
steps = 10
forecaster = ForecasterAutoreg(LGBMRegressor(verbose=-1), lags=3)
forecaster.fit(y=y, exog=exog)
last_window_values, exog_values, _ = (
    forecaster._create_predict_inputs(steps=steps, exog=exog_predict)
)

sampled_residuals = {
                k: v[rng.integers(low=0, high=len(v), size=steps)]
                for k, v in forecaster.in_sample_residuals_by_bin_.items()
            }

predictions = forecaster._recursive_predict(
                    steps                = steps,
                    last_window_values   = last_window_values,
                    exog_values          = exog_values,
                    residuals            = sampled_residuals,
                    use_binned_residuals = True,
                )

expected = np.array([
            0.951684, 0.115618, 0.160797, 0.551315, 0.747326, 0.634401,
            0.099628, 0.47903, 0.387516, 0.581912
           ])

np.testing.assert_array_almost_equal(predictions, expected)

In [83]:
forecaster.in_sample_residuals_by_bin_

{0: array([ 0.05425458, -0.27934073,  0.19912894,  0.07134645, -0.10508544,
         0.43446198]),
 1: array([-0.04434585, -0.09619266, -0.04132642, -0.27857404,  0.0396593 ,
         0.02900846]),
 2: array([-0.0980108 , -0.05583897]),
 3: array([-0.05759753, -0.31367804,  0.34573851,  0.22076204,  0.10733023,
        -0.18073437,  0.12025967, -0.38807488, -0.2457144 ]),
 4: array([-0.07509307]),
 5: array([-0.09524194,  0.01522903,  0.11605256, -0.01651172, -0.09403693,
         0.00113672]),
 6: array([ 0.02202285,  0.19975779, -0.22980421,  0.3698709 ]),
 7: array([ 0.44972244,  0.111644  , -0.21375628]),
 8: array([ 0.13162811,  0.1594852 ,  0.12707902, -0.49573592,  0.35631916,
         0.01753017]),
 9: array([-0.06898285, -0.37227099, -0.17418291,  0.32070297])}

In [85]:
for k, v in forecaster.in_sample_residuals_by_bin_.items():
    print(k, len(v))

0 6
1 6
2 2
3 9
4 1
5 6
6 4
7 3
8 6
9 4


In [44]:
for k, v in forecaster.out_sample_residuals_by_bin_.items():
    print(k, len(v))

0 1000
1 1000
2 1000
3 1000
4 1000
5 1000
6 1000
7 1000
8 1000
9 1000


In [31]:
forecaster._binning_in_sample_residuals(
    y_true = y_true,
    y_pred = y_true,

)
print(forecaster.binner.bin_edges_)
forecaster.binner_intervals_

[-1.49443809 -0.18000684  0.1663009   0.35273434  0.63528567  0.90802292
  1.12086466  1.42902786  1.77815393  2.32329658  3.93117714]


{0.0: (-1.4944380855730266, -0.1800068383929535),
 1.0: (-0.1800068383929535, 0.16630089975318266),
 2.0: (0.16630089975318266, 0.3527343445621452),
 3.0: (0.3527343445621452, 0.6352856739829892),
 4.0: (0.6352856739829892, 0.9080229150474954),
 5.0: (0.9080229150474954, 1.1208646636265782),
 6.0: (1.1208646636265782, 1.4290278627173163),
 7.0: (1.4290278627173163, 1.778153929057584),
 8.0: (1.778153929057584, 2.323296575143496),
 9.0: (2.323296575143496, 3.931177135195687)}

In [26]:
_ = forecaster.binner.fit(y_true)
print(forecaster.binner.bin_edges_)
forecaster.binner.intervals_

[-17.47117318  -2.32095544   1.84659725   4.87235529   8.0943841
  10.60186959  12.59386984  14.52902809  18.08644084  22.34716771
  37.45662982]


{0.0: (-17.471173182613413, -2.3209554390889844),
 1.0: (-2.3209554390889844, 1.846597248735215),
 2.0: (1.846597248735215, 4.87235528938399),
 3.0: (4.87235528938399, 8.094384096101859),
 4.0: (8.094384096101859, 10.60186958630161),
 5.0: (10.60186958630161, 12.59386983776179),
 6.0: (12.59386983776179, 14.529028087323077),
 7.0: (14.529028087323077, 18.086440838165675),
 8.0: (18.086440838165675, 22.347167711175327),
 9.0: (22.347167711175327, 37.45662981882582)}

In [27]:
_ = forecaster.binner.fit(y_true)
print(forecaster.binner.bin_edges_)
forecaster.binner.intervals_

[-17.47117318  -2.32095544   1.84659725   4.87235529   8.0943841
  10.60186959  12.59386984  14.52902809  18.08644084  22.34716771
  37.45662982]


{0.0: (-17.471173182613413, -2.3209554390889844),
 1.0: (-2.3209554390889844, 1.846597248735215),
 2.0: (1.846597248735215, 4.87235528938399),
 3.0: (4.87235528938399, 8.094384096101859),
 4.0: (8.094384096101859, 10.60186958630161),
 5.0: (10.60186958630161, 12.59386983776179),
 6.0: (12.59386983776179, 14.529028087323077),
 7.0: (14.529028087323077, 18.086440838165675),
 8.0: (18.086440838165675, 22.347167711175327),
 9.0: (22.347167711175327, 37.45662981882582)}